In [2]:
import torch
import torch.nn as nn

In [3]:
# 이전 step의 context(어텐션 결과)를 다음 step 입력에 붙여 LSTM에 넣는 Input Feeding 디코더
class InputFeedingDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(InputFeedingDecoder, self).__init__()
        self.hidden_size = hidden_size                              # 디코더 은닉 차원 저장
        self.embedding = nn.Embedding(output_size, input_size)      # 출력 토큰 ID -> 입력 임베딩(input_size)
        self.lstm = nn.LSTM(input_size + hidden_size, hidden_size)  # (임베딩 + context) -> hidden_size LSTM
        self.fc = nn.Linear(hidden_size, output_size)               # hidden -> 어휘 크기(output_size) 로짓

    # 입력 토큰/이전 hidden/이전 context를 받아 다음 토큰 로짓과 hidden을 반환
    def forward(self, input, hidden, context):
        embedded = self.embedding(input).unsqueeze(0)  # (B,) -> (1, B, input_size)
        lstm_input = torch.cat((embedded, context.unsqueeze(0)), dim=2)  # 임베딩과 context를 feature 차원으로 결합
        output, hidden = self.lstm(lstm_input, hidden) # LSTM 1-step 수행 -> output(1, B, H), hidden((1, B, H),(1, B, H))
        output = self.fc(output.squeeze(0))            # (1, B, H) -> (B, H) -> (B, output_size)로 변환
        return output, hidden                          # 로짓과 다음 hidden값 반환

In [ ]:
# Input Feeding Decoder 더미 입력으로 1-step 실행 확인
decoder = InputFeedingDecoder(input_size=10, hidden_size=20, output_size=30)    # 디코더 생성 (임베딩:10, hidden:20, vocab:30)
hidden = (torch.zeros(1, 1, 20), torch.zeros(1, 1, 20))  # 초기 hidden state(h, c) : (num_layers, B, H)
context = torch.zeros(1, 20)                             # 이전 context 벡터 (B, H)
input_token = torch.tensor([5])                          # 입력 토큰 ID (B,)

output, hidden = decoder(input_token, hidden, context)   # 디코더 1-step 실행
output.shape, hidden[0].shape, hidden[1].shape           # output : (B, vocab_size), hidden(h), hidden(c) : (num_layers, B, H)

(torch.Size([1, 30]), torch.Size([1, 1, 20]), torch.Size([1, 1, 20]))